In [29]:
import os
import librosa
import pandas as pd
import numpy as np
import csv
import keras
import tkinter as tk
from tkinter import *
from mutagen.mp3 import MP3
from pygame import mixer
from pydub import AudioSegment

import warnings
warnings.filterwarnings('ignore')
from keras.models import load_model
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [34]:
def detectEmotion(path):
    songname = f'{path}'
    print (songname)
    base=os.path.basename(path)
    print (base)
    name=os.path.splitext(base)[0]
    print (name)
    header = 'chroma_stft spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
    for i in range(1, 21):
         header += f' mfcc{i}'

    header = header.split()

    file = open('song_data.csv', 'w', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(header)
        
    #feature extraction using Librosa library
    y, sr = librosa.load(songname, mono=True, duration=30)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    to_append = f'{np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}' 
    print(to_append)
    for e in mfcc:
        to_append += f' {np.mean(e)}'
    file = open('song_data.csv', 'a', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(to_append.split())

    model = load_model('my_model2.h5')
    data_from_csv = pd.read_csv("song_data.csv")
    data = data_from_csv
#     print (data)
    data.shape
    scaler = joblib.load("scaler2.save") 
    X = scaler.transform(np.array(data.iloc[:,:], dtype = float))
    print(X)
    ynew = model.predict(np.array(X))
    
    #ynew = model.predict(np.array(data))
    print (ynew)
    if np.argmax(ynew)==0:
        print ("Calm")
        return "calm"
    elif np.argmax(ynew)==1:
        print ("Happy")
        return "happy"
    elif np.argmax(ynew)==2:
        print ("Sad")
        return "sad"


detectEmotion("E:/mp3/SLANDER - Love Is Gone ft. Dylan Matthew (Acoustic) - Lyrics.mp3")

E:/mp3/SLANDER - Love Is Gone ft. Dylan Matthew (Acoustic) - Lyrics.mp3
SLANDER - Love Is Gone ft. Dylan Matthew (Acoustic) - Lyrics.mp3
SLANDER - Love Is Gone ft. Dylan Matthew (Acoustic) - Lyrics
0.24715054035186768 1168.3372893646433 1343.8130849903005 1933.0966441623937 0.07452978364454334
[[-0.92157222 -0.94688686 -1.31007018 -1.17290409 -0.02525416 -0.34057913
   1.51182241  0.32350962 -2.51430024  0.48521616  1.1129692   0.27049657
   0.29130457 -0.63448346  0.21416597  1.26218321  0.4481586  -0.2759423
  -0.18909878  0.97412459  1.05383254  0.19976235  0.70140228  0.30125076
   0.56578531]]
[[0.04859083 0.0023382  0.949071  ]]
Sad


'sad'